<a href="https://colab.research.google.com/github/ChallaHarika23/Aimlprogram/blob/main/Module3_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
import re
import numpy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bs4 import BeautifulSoup

def cleanText(text, lemmatize, stemmer):

    if isinstance(text, float):
        text = str(text)
    if isinstance(text, numpy.int64):
        text = str(text)
    try:
        text = text.decode()
    except AttributeError:
        pass

    soup = BeautifulSoup(text, "lxml")
    text = soup.get_text()
    text = re.sub(r"[^A-Za-z]", " ", text)
    text = text.lower()


    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()

        def gettag(tag):
            if tag.startswith('J'):
                return wordnet.ADJ
            elif tag.startswith('V'):
                return wordnet.VERB
            elif tag.startswith('N'):
                return wordnet.NOUN
            elif tag.startswith('R'):
                return wordnet.ADV
            else:
                return ''

        text_res = []
        tokens = word_tokenize(text)
        tagged = pos_tag(tokens)
        for t in tagged:
            try:
                text_res.append(wordnet_lemmatizer.lemmatize(t[0], gettag(t[1][:2])))
            except:
                text_res.append(wordnet_lemmatizer.lemmatize(t[0]))
        return text_res

    if stemmer:
        text_res = []
        tokens = word_tokenize(text)
        snowball_stemmer = SnowballStemmer('english')
        for t in tokens:
            text_res.append(snowball_stemmer.stem(t))
        return text_res

In [3]:
ex_text = "Discovery"
text_result = cleanText(ex_text, lemmatize=False, stemmer=True)
text_result = " ".join(str(x) for x in text_result)
print(ex_text)
print(text_result)
text_result = cleanText(ex_text, lemmatize=True, stemmer=False)
text_result = " ".join(str(x) for x in text_result)
print(text_result)

Discovery
discoveri
discovery


## Section 1.2: BAG OF WORDS

In [4]:
def createBagOfWords(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = CountVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer = CountVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    bagwords_train = vectorizer.fit_transform(clean_train).toarray()
    bagwords_test = vectorizer.transform(clean_test).toarray()
    return bagwords_train, bagwords_test


## Section 1.3: TF-IDF

In [5]:
def createTFIDF(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = TfidfVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer =  TfidfVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    tfidf_train = vectorizer.fit_transform(clean_train).toarray()
    tfidf_test = vectorizer.transform(clean_test).toarray()
    return tfidf_train, tfidf_test

## Section 2: UNDERSTANDING THE DATA : A REVIEWS DATASET

In [6]:
from google.colab import files
uploaded = files.upload()

Saving reviews.csv to reviews (1).csv


In [7]:
import pandas as pd
data = pd.read_csv('reviews.csv')

In [8]:
data=data.dropna()
data

,sentence,sentiment
0,Not sure who was more lost - the flat characte...,0
1,Attempting artiness with black & white and cle...,0
2,Very little music or anything to speak of.,0
3,The best scene in the movie was when Gerardo i...,1
4,"The rest of the movie lacks art, charm, meanin...",0
...,...,...
950,I just got bored watching Jessice Lange take h...,0
951,"Unfortunately, any virtue in this film's produ...",0
952,"In a word, it is embarrassing.",0
953,Exceptionally bad!,0


In [9]:
data.to_csv('reviews.csv', index=False)

## Section 3: KNN MODEL

In [10]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict


def knn():

    data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(data["sentence"], data["sentiment"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')

knn()

<ipython-input-2-ceacd1606313>:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
KNN with BOW accuracy = 62.30366492146597%


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Cross Validation Accuracy: 0.62
[0.60784314 0.58431373 0.66141732]




In [11]:
def tfidf_knn():

    data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(data["sentence"], data["sentiment"],
                                                        test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    #print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2,
                                         metric='cosine', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

tfidf_knn()

<ipython-input-2-ceacd1606313>:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with TFIDF accuracy = 70.15706806282722%
Cross Validation Accuracy: 0.73
[0.7254902  0.74117647 0.72834646]


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


(array(['1', '1', '1', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1',
        '0', '0', '1', '1', '1', '0', '1', '1', '0', '0', '1', '1', '1',
        '1', '1', '0', '0', '0', '1', '1', '0', '0', '0', '0', '1', '0',
        '0', '1', '1', '0', '1', '0', '0', '0', '0', '1', '0', '0', '1',
        '0', '0', '1', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0',
        '0', '1', '0', '0', '1', '0', '0', '1', '1', '0', '0', '0', '1',
        '1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '1',
        '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '0', '1', '1',
        '1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '1',
        '0', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1',
        '0', '1', '0', '0', '1', '1', '1', '0', '0', '0', '1', '0', '1',
        '1', '0', '0', '0', '1', '1', '1', '0', '0', '1', '1', '0', '0',
        '1', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '1',
        '0', '0', '1', '0', '0', '1', '0', '1', '0'

## TASK 1

In [12]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from functools import partial

def knn():

    data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(data["sentence"], data["sentiment"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=8, weights='uniform', leaf_size=25, p=2,algorithm='auto',
                        metric='minkowski', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')

knn()

<ipython-input-2-ceacd1606313>:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
KNN with BOW accuracy = 60.20942408376963%
Cross Validation Accuracy: 0.63
[0.62745098 0.61176471 0.64173228]




/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


In [13]:
def tfidf_knn():

    data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(data["sentence"], data["sentiment"],
                                                        test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    #print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=10, weights='distance', algorithm='brute', leaf_size=24, p=2,
                                         metric='minkowski', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

tfidf_knn()

<ipython-input-2-ceacd1606313>:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with TFIDF accuracy = 73.29842931937172%
Cross Validation Accuracy: 0.74
[0.7254902  0.76862745 0.72440945]


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


(array(['1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '1',
        '1', '0', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '1',
        '1', '1', '0', '0', '0', '1', '1', '0', '0', '0', '0', '1', '1',
        '0', '1', '1', '0', '1', '0', '0', '1', '0', '1', '0', '1', '1',
        '0', '0', '1', '0', '0', '1', '0', '0', '1', '1', '1', '1', '0',
        '1', '1', '1', '0', '1', '0', '1', '1', '1', '1', '0', '0', '1',
        '1', '1', '0', '1', '0', '0', '0', '1', '1', '0', '1', '1', '1',
        '1', '1', '1', '0', '0', '1', '1', '0', '0', '1', '0', '1', '1',
        '1', '1', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '1',
        '0', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1',
        '0', '1', '0', '0', '1', '1', '1', '1', '0', '1', '1', '0', '1',
        '1', '0', '0', '0', '1', '1', '1', '0', '1', '1', '1', '0', '1',
        '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '0', '1',
        '0', '1', '1', '0', '1', '1', '0', '1', '0'

## Section 4: SPAM TEXT DATASET

In [14]:
from google.colab import files
upload=files.upload()

Saving spam.csv to spam (1).csv


In [15]:
data1 = pd.read_csv('spam.csv', error_bad_lines=False)
data1

<ipython-input-15-d89421125aa1>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data1 = pd.read_csv('spam.csv', error_bad_lines=False)


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ã¼ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [16]:
data1['Category'] = data1['Category'].map({'ham': 0, 'spam': 1})
data1.head(5)

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [17]:
len(data1)

5572

In [18]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

def bow_knn():

    data = pd.read_csv('spam.csv')
    data['Category'] = data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(data["Message"], data["Category"], test_size=0.4, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')
    return predicted, y_test

In [19]:
predicted, y_test = bow_knn()

<ipython-input-2-ceacd1606313>:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with BOW accuracy = 90.93764019739794%
Cross Validation Accuracy: 0.90
[0.89327354 0.90125673 0.90394973]




In [20]:
def tfidf_knn():

    data = pd.read_csv('spam.csv')
    data['Category'] = data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(data["Message"], data["Category"], test_size=0.4, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2, metric='cosine', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

In [21]:
predicted, y_test = tfidf_knn()

<ipython-input-2-ceacd1606313>:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with TFIDF accuracy = 97.89143113503813%
Cross Validation Accuracy: 0.96
[0.96591928 0.96140036 0.96050269]


## TASK - 2: Tweak the models below and see results with different parameters and distance metrics.

In [22]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

def bow_knn():

    data = pd.read_csv('spam.csv')
    data['Category'] = data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(data["Message"], data["Category"], test_size=0.4, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=14, weights='distance', algorithm='auto', leaf_size=45, p=2, metric='cosine', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')
    return predicted, y_test

bow_knn()

<ipython-input-2-ceacd1606313>:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with BOW accuracy = 97.30820995962316%
Cross Validation Accuracy: 0.96
[0.96143498 0.95870736 0.96409336]




(array([1, 0, 1, ..., 0, 0, 0]),
 2095    1
 5343    0
 564     1
 3849    0
 3317    0
        ..
 5001    0
 5024    0
 295     0
 1180    0
 1135    0
 Name: Category, Length: 2229, dtype: int64)

In [23]:
def tfidf_knn():

    data = pd.read_csv('spam.csv')
    data['Category'] = data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(data["Message"], data["Category"], test_size=0.4, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=4, weights='distance', algorithm='brute', leaf_size=30, p=4, metric='minkowski', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

tfidf_knn()

<ipython-input-2-ceacd1606313>:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with TFIDF accuracy = 92.91161956034097%
Cross Validation Accuracy: 0.90
[0.89955157 0.90035907 0.91023339]


(array([1, 0, 1, ..., 0, 0, 0]),
 2095    1
 5343    0
 564     1
 3849    0
 3317    0
        ..
 5001    0
 5024    0
 295     0
 1180    0
 1135    0
 Name: Category, Length: 2229, dtype: int64)

## Questions to Think About and Answer

## 1. Why does the TF-IDF approach generally result in a better accuracy than Bag-of-Words ?

Word Importance Across Documents,Handling Stop Words,Handling Stop Words,the mentioned tasks are performed better using TFIDF when compared to BOW.
TF-IDF is a more sophisticated approach that takes into account both the local importance of a term within a document and its global importance across the entire corpus. This makes TF-IDF a more effective method for representing documents and is often preferred over the simpler Bag-of-Words model in various natural language processing tasks.


## 2.Can you think of techniques that are better than both BoW and TF-IDF ?


1.Word embeddings, such as Word2Vec, GloVe, and FastText, represent words as dense vectors in a continuous vector space. These embeddings capture semantic relationships between words and are trained to place similar words close to each other in the vector space. Word embeddings can capture more nuanced relationships and context compared to the simple word frequency-based approaches.

2.BERT (Bidirectional Encoder Representations from Transformers)is a pre-trained language model based on transformers. It captures bidirectional contextual information and can be fine-tuned for specific tasks. BERT embeddings have been shown to outperform traditional methods in a wide range of natural language processing tasks, including sentiment analysis, named entity recognition, and question answering.

### 3.Read about Stemming and Lemmatization from the resources given below. Think about the pros/cons of each.

### Stemming Pros:

1.Stemming is a simpler and faster process compared to lemmatization. It involves removing prefixes or suffixes from words to obtain their root forms.

2.Stemming can help in reducing the dimensionality of the feature space by collapsing different inflections of a word into a common base form.

3.stemming is computationally less expensive, making it suitable for large datasets or real-time applications.

### Cons:
Stemming might not always produce linguistically correct words, as it applies generic rules without considering the context or the actual meaning of the words.

### Lemmatization pros
1.Lemmatization aims to produce valid words and maintains the linguistic accuracy of the text. It considers the context and part of speech of the word, providing a more accurate base form.

2.Lemmatization retains more semantic information compared to stemming.

3.Lemmatization takes into account the grammatical context of words, considering variations based on tense, gender, number, and other linguistic features.

### Cons

1.Lemmatization is a more complex and computationally intensive process compared to stemming.

2.lemmatization may be slower than stemming, which could be a consideration in real-time applications or large-scale processing.